###Criando um ReAct Agent do Zero (I) - JonasJunior

In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 2.6 MB/s eta 0:00:00


In [ ]:
# Configuração da chave de API da Groq
import os
os.environ['GROQ_API_KEY'] = "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"

_________________________________________________________________________________________________

In [ ]:
#Redefinição da classe 'Agent' para meu exemplo
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile", messages=self.messages
        )
        return completion.choices[0].message.content

In [ ]:
#Definição de 'recommend_movie' e 'recommend_restaurant'.
#'recommend_movie' Recomenda um filme com base no gênero.
#'recommend_restaurant' Recomenda um restaurante com base na culinaria e localizacao.

def recommend_movie(genre: str) -> str:
    match genre.lower():
        case "ficção científica":
            return "Interestelar"
        case "ação":
            return "John Wick"
        case "drama":
            return "Forrest Gump"
        case _:
            return "Gênero não reconhecido."

def recommend_restaurant(cuisine: str, location: str) -> str:
    if cuisine.lower() == "italiano" and location.lower() == "são paulo":
        return "Famiglia Mancini"
    elif cuisine.lower() == "japonês" and location.lower() == "são paulo":
        return "Kinoshita"
    else:
        return "Nenhum restaurante encontrado para os critérios fornecidos."

In [ ]:
#Novo Prompt para os novos exemplos que define o sistema para ser recomendado incluindo ações das funções recomend_movie e recommend_restaurant

system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

recommend_movie:
e.g. recommend_movie: Ficção Científica
Recomenda um filme com base no gênero.

recommend_restaurant:
e.g. recommend_restaurant: Italiano, São Paulo
Recomenda um restaurante com base na culinária e localização.

Example session:

Question: Recomende um filme de ficção científica e um restaurante italiano em São Paulo.
Thought: Preciso recomendar um filme de ficção científica e um restaurante italiano em São Paulo.
Action: recommend_movie: Ficção Científica
PAUSE

You will be called again with this:

Observation: Recomendo o filme 'Interestelar'.

Thought: Agora preciso recomendar um restaurante italiano em São Paulo.
Action: recommend_restaurant: Italiano, São Paulo
PAUSE

You will be called again with this:

Observation: Recomendo o restaurante 'Famiglia Mancini'.

Answer: Recomendo o filme 'Interestelar' e o restaurante 'Famiglia Mancini' em São Paulo.

Now it's your turn:
""".strip()

In [ ]:
recommendation_agent = Agent(client=client, system=system_prompt)

In [ ]:
# Função para executar o loop do agente

def agent_loop(max_iterations=10, query: str = ""):
    tools = ["recommend_movie", "recommend_restaurant"]
    next_prompt = query
    i = 0

    while i < max_iterations:
        i += 1
        result = recommendation_agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            args = action[0][1].split(", ")

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{args[0]}', '{args[1]}')" if chosen_tool == "recommend_restaurant" else f"{chosen_tool}('{args[0]}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


In [ ]:
agent_loop(query="Recomende um filme de ficção científica e um restaurante italiano em São Paulo.") #Inicia o loop do agente com uma consulta para recomendar um filme e um restaurant

Thought: Preciso recomendar um filme de ficção científica e um restaurante italiano em São Paulo. Para começar, vou recomendar um filme de ficção científica, pois é um gênero popular e com muitas opções interessantes.

Action: recommend_movie: Ficção Científica
PAUSE
Observation: Interestelar
Thought: Agora que tenho o filme de ficção científica recomendado, que é "Interestelar", preciso recomendar um restaurante italiano em São Paulo. São Paulo é conhecida por sua diversidade gastronômica e tem muitos restaurantes italianos excelentes.

Action: recommend_restaurant: Italiano, São Paulo
PAUSE
Observation: Famiglia Mancini
Thought: Com o filme "Interestelar" e o restaurante "Famiglia Mancini" recomendados, posso agora fornecer a resposta completa.

Answer: Recomendo o filme 'Interestelar' e o restaurante 'Famiglia Mancini' em São Paulo.
